In [17]:
import pandas as pd

off_train = pd.read_csv('data/ccf_offline_stage1_train.csv',parse_dates=["Date"],header=0)
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']

off_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv',header=0)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']

on_train = pd.read_csv('data/ccf_online_stage1_train.csv',parse_dates=["Date"],header=0)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']

# 【dataset3】20160701~20160731 (113640)   【feature3】20160315~20160630  （测试集）
dataset3 = off_test.dropna(how='any')
feature3 = off_train[((off_train.date >= '20160315') & (off_train.date <= '20160630'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160315') 
                      &  (off_train.date_received <= '20160630'))
                   ].dropna(how='any')
# 【dataset2】20160515~20160615 (258446)   【feature2】20160201~20160514  （训练集2）
dataset2 = off_train[((off_train.date_received >= '20160515') & (off_train.date_received <= '20160615'))].dropna(how='any')
feature2 = off_train[((off_train.date >= '20160201') & (off_train.date <= '20160514'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160201') 
                      &  (off_train.date_received <= '20160514'))
                   ].dropna(how='any')
# 【dataset1】20160414~20160514 (138303)   【feature1】20160101~20160413  （训练集1）
dataset1 = off_train[((off_train.date_received >= '20160414') & (off_train.date_received <= '20160514'))].dropna(how='any')
feature1 = off_train[((off_train.date >= '20160101') & (off_train.date <= '20160413'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160101') 
                      &  (off_train.date_received <= '20160413'))
                   ].dropna(how='any')

In [18]:
from datetime import date
import numpy as np
def is_firstlastone(x):
    if x == 0:
         return 1
    elif x > 0:
        return 0
    else:
        return -1  # those only receive once

def get_day_gap_before(s):
    date_received, dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        af_date = date(int(date_received[0:4]), int(date_received[4:6]), int(date_received[6:8]))
        bf_date =  date(int(d[0:4]),int(d[4:6]),int(d[6:8]))
        this_gap = (af_date - bf_date).days
        if this_gap > 0:
            gaps.append(this_gap)
    if len(gaps) == 0:
        return -1
    else:
        return min(gaps)


def get_day_gap_after(s):
    date_received, dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        bf_date = date(int(date_received[0:4]), int(date_received[4:6]), int(date_received[6:8]))
        af_date =  date(int(d[0:4]),int(d[4:6]),int(d[6:8]))
        this_gap = (af_date - bf_date).days
        if this_gap > 0:
            gaps.append(this_gap)
    if len(gaps) == 0:
        return -1
    else:
        return min(gaps)

In [19]:
def dealOtherFeature(dataset,file_name):
    #this_month_user_receive_all_coupon_count 这个月用户收取的所有优惠券数目 
    t = dataset[['user_id']]
    t['this_month_user_receive_all_coupon_count'] = 1
    t = t.groupby('user_id').agg('sum').reset_index()
    t.ix[:5]
    #this_month_user_receive_same_coupon_count 这个月用户收到的相同优惠券的数量 
    t1 = dataset[['user_id','coupon_id']]
    t1['this_month_user_receive_same_coupon_count'] = 1
    t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()
    t1.ix[:5]
    #this_month_user_receive_same_coupon_lastone 这个月优惠券最远接受时间  
    #this_month_user_receive_same_coupon_firstone 这个月优惠券最近接受时间 
    t2 = dataset[['user_id','coupon_id','date_received']]
    t2.date_received = t2.date_received.astype('str')
    t2 = t2.groupby(['user_id', 'coupon_id'])['date_received'].agg(lambda x: ':'.join(x)).reset_index()
    t2['receive_number'] = t2.date_received.apply(lambda s: len(s.split(':')))
    t2 = t2[t2.receive_number > 1]
    t2['max_date_received'] = t2.date_received.apply(lambda s: max([int(d) for d in s.split(':')]))
    t2['min_date_received'] = t2.date_received.apply(lambda s: min([int(d) for d in s.split(':')]))
    t2 = t2[['user_id','coupon_id','max_date_received','min_date_received']]
    t2.ix[:5]
    
    t3 = dataset[['user_id','coupon_id','date_received']]
    t3 = pd.merge(t3,t2,on=['user_id','coupon_id'],how='left').dropna(how='any')
    t3.date_received = t3.date_received.astype('float')
    t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received
    t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received - t3.min_date_received
    t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(
            is_firstlastone)
    t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(
            is_firstlastone)
    t3 = t3[['user_id', 'coupon_id', 'date_received', 'this_month_user_receive_same_coupon_lastone',
                 'this_month_user_receive_same_coupon_firstone']]
    t3.ix[:5]
    
    #this_day_user_receive_all_coupon_count 一天内用户接收到所有优惠券的数量  
    t4 = dataset[['user_id','date_received']]
    t4['this_day_user_receive_all_coupon_count'] = 1
    t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()
    t4.ix[:5]
    
    #this_day_user_receive_same_coupon_count 一天内用户接收到相同优惠券的数量
    t5 = dataset[['user_id','coupon_id','date_received']]
    t5['this_day_user_receive_same_coupon_count'] = 1
    t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()
    t5.ix[:5]
    
    #day_gap_after  用户下一次领取优惠券的时间间隔 (同一优惠券)
    #day_gap_before 用户上一次领取优惠券的时间间隔 
    t6 = dataset[['user_id','coupon_id','date_received']]
    t6.date_received = t6.date_received.astype('str')
    t6 = t6.groupby(['user_id', 'coupon_id'])['date_received'].agg(lambda x: ':'.join(x)).reset_index()
    t6.rename(columns={'date_received': 'dates'}, inplace=True)
     
    t7 = dataset[['user_id','coupon_id','date_received']]
    t7 = pd.merge(t7,t6,on=['user_id','coupon_id'],how='left').dropna(how='any')
    t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates

    t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
    t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
    t7 = t7[['user_id', 'coupon_id', 'date_received', 'day_gap_before', 'day_gap_after']]
    t7.ix[:5]
    
    # 合并所有的特征
    other_feature3 = pd.merge(t1,t,on='user_id').dropna(how='any')
    other_feature3 = pd.merge(other_feature3,t3,on=['user_id','coupon_id']).dropna(how='any')
    other_feature3 = pd.merge(other_feature3,t4,on=['user_id','date_received']).dropna(how='any')
    other_feature3 = pd.merge(other_feature3,t5,on=['user_id','coupon_id','date_received']).dropna(how='any')
    other_feature3 = pd.merge(other_feature3,t7,on=['user_id','coupon_id','date_received']).dropna(how='any')

    other_feature3.to_csv(file_name)
    return other_feature3

In [20]:
other_feature3 = dealOtherFeature(dataset3,'data/other_feature3.csv')
other_feature3.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

,user_id,coupon_id,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
1,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
2,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
3,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
4,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1
5,5568010,8596,1,2,20160722.0,1,1,2,2,-1,-1


In [21]:
other_feature2 = dealOtherFeature(dataset2,'data/other_feature2.csv')
other_feature2.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,user_id,coupon_id,date_received,day_gap_before,day_gap_after


In [22]:
other_feature1 = dealOtherFeature(dataset1,'data/other_feature1.csv')
other_feature1.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,user_id,coupon_id,date_received,day_gap_before,day_gap_after
